In [69]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
#from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [70]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("final_dataset.csv")
print(data_initial.shape)

(3569, 694)


In [72]:
data_no_na = data_initial.dropna(axis = 1) # data should already have no NAs due to numerical imputation
print(data_no_na.shape)
i = 0
for col in data_no_na.columns:
   print("" + str(i) + ": " + col)
   i = i + 1

include_col_indices = [0, 48]  # Start with individual indices

# Add ranges using loops
include_col_indices.extend(range(63, 139))
include_col_indices.extend(range(159, 279))
include_col_indices.extend(range(299, 343))
include_col_indices.extend(range(623, 691))

print(include_col_indices)
print(len(include_col_indices))

# Subset columns from the NumPy array
subset_data = data_no_na.iloc[:, include_col_indices]

# Convert the subsetted array to a DataFrame
df_subset = pd.DataFrame(subset_data)

# Select columns of number data types
data = df_subset.select_dtypes(include=['number'])
data_removed = data[data['season'] == 20212022]
print(data_removed.shape)
data = data[data['season'] != 20212022]
data_unnorm = data


from sklearn import preprocessing

vals = data.values #returns a numpy array
scaler = preprocessing.StandardScaler()
scaler_fit= scaler.fit(vals)
scaler_transform = scaler_fit.transform(vals)
data = pd.DataFrame(vals_scaled, columns = data.columns)
#print(data)
print(data.columns)

y = data['FA_zone_time']
x = data.loc[:, data.columns != 'FA_zone_time']
x = x.loc[:, x.columns != 'FA_zone_time']
X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.6, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, train_size = 0.5, test_size=0.5, random_state=45)

# Above two lines in unison accomplish a 60-20-20 split for train-validation-test

seasons_train = data_initial.iloc[X_train.index]['season']
seasons_val = data_initial.iloc[X_val.index]['season']
seasons_test = data_initial.iloc[X_test.index]['season']

print(seasons_test)

player_ids_train = data_initial.iloc[X_train.index]['event_player_1']
player_ids_val = data_initial.iloc[X_val.index]['event_player_1']
player_ids_test = data_initial.iloc[X_test.index]['event_player_1']
print(player_ids_test)

opposing_player_ids_train = data_initial.iloc[X_train.index]['event_player_2']
opposing_player_ids_val = data_initial.iloc[X_val.index]['event_player_2']
opposing_player_ids_test = data_initial.iloc[X_test.index]['event_player_2']
print(opposing_player_ids_test)

(3569, 691)
0: season
1: game_id
2: game_date
3: session
4: event_index
5: game_period
6: game_seconds
7: clock_time
8: event_type
9: event_description
10: event_zone
11: event_team
12: event_player_1
13: event_player_2
14: event_length
15: coords_x
16: coords_y
17: home_goalie
18: away_goalie
19: home_team
20: away_team
21: home_score
22: away_score
23: game_score_state
24: game_strength_state
25: home_zone
26: event_distance
27: event_angle
28: home_zonestart
29: face_index
30: pen_index
31: shift_index
32: pbp
33: teams
34: last_faceoff_winner
35: last_faceoff_winner_home
36: last_faceoff_winner_zone
37: event_team_relative_to_faceoff
38: event_zone_relative_to_faceoff_winner
39: last_faceoff_winner_faceoff_zone
40: event_team_positioning_last_faceoff
41: last_faceoff_loser_faceoff_zone
42: start_FA
43: next_stoppage
44: next_neutral_zone_event
45: next_zone_exit
46: is_FA
47: end_FA
48: FA_zone_time
49: Pos_home_on_3
50: Pos_home_on_4
51: Pos_home_on_5
52: Pos_away_on_1
53: Pos_awa

In [74]:
device_selection = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_selection)
print(device_selection)
print(device)

if torch.cuda.is_available():
        torch.cuda.empty_cache()

train_batch_size = 256
val_batch_size = 256
test_batch_size = 256

cpu
cpu


In [75]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.df.iloc[idx, -1:].values, dtype=torch.float32)
        return x, y

train_df =  pd.concat([X_train, y_train], axis=1)  # pandas DataFrame containing training data
valid_df = pd.concat([X_val, y_val], axis=1) # pandas DataFrame containing validation data
test_df = pd.concat([X_test, y_test], axis=1)  # pandas DataFrame containing test data
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

# Credit to ChatGPT for generating this cell of code in response to the following prompt: design a dataloader that accepts the output of a pandas train-validation-test split. Basically write code for a PyTorch dataloader that accepts pandas dataframes as arguments

In [76]:
class VanillaNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(VanillaNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(self.input_size, 1000)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.batch_norm_1000 = nn.BatchNorm1d(1000)
        self.batch_norm_500 = nn.BatchNorm1d(500)
        self.batch_norm_400 = nn.BatchNorm1d(400)
        self.batch_norm_300 = nn.BatchNorm1d(300)
        self.batch_norm_200 = nn.BatchNorm1d(200)
        self.batch_norm_100 = nn.BatchNorm1d(100)

        self.fc2 = nn.Linear(1000, 1000)

        self.fc3 = nn.Linear(1000, 1000)

        self.fc4 = nn.Linear(1000, 500)

        self.fc5 = nn.Linear(500, 500)

        self.fc6 = nn.Linear(500, 400)

        self.fc7 = nn.Linear(400, 300)

        self.fc8 = nn.Linear(300, 200)

        self.fc9 = nn.Linear(200, 100)

        self.fc10 = nn.Linear(100, 1)

    def forward(self, x):
        #print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_1000(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_500(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_500(x)

        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_400(x)

        x = self.fc7(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_300(x)

        x = self.fc8(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_200(x)

        x = self.fc9(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm_100(x)

        x = self.fc10(x)
        
        return x


In [77]:
num_epochs = 50
trial_count = 50

learning_rate = 0.01 # in last optimization, best lr ended up being 0.0016
loss_func = nn.MSELoss()

def train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device):
    training_losses = np.array([])
    validation_losses = np.array([])
    for epoch in range(num_epochs):
        print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
        ########################### Training #####################################
        model.train(True)
        train_loss = 0
        count = 0
        for batch, (X, y) in enumerate(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            optimizer.zero_grad()
            prediction = model(X).to(device)
            loss = loss_func(prediction, y.view(-1, 1))
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.float().item()
            count += train_batch_size
        training_losses = np.append(training_losses, train_loss / count)
        print("Training loss:", train_loss / count)
        #model.train(False)

        ########################### Validation #####################################
        model.eval()
        val_loss = 0
        count = 0
        with torch.no_grad():
            for batch, (X, y) in enumerate(valid_loader):
                X = X.to(device)
                y = y.to(device)
                prediction = model(X).to(device)
                loss = loss_func(prediction, y.view(-1, 1))
                val_loss += loss.float().item()
                count += val_batch_size
        validation_losses = np.append(validation_losses, val_loss / count)
        print("Validation loss:", val_loss / count)
    return validation_losses[-1]

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hidden_size = trial.suggest_int("hidden_size", 300, 600)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    print("learning rate:", lr)
    print("weight_decay:", weight_decay)
    print("hidden_size:", hidden_size)
    print("dropout_rate:", dropout_rate)

    model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

input_dim = 309
output_dim = 1
hidden_dim = 300
dropout = 0.1
model = VanillaNeuralNet(input_dim, hidden_dim, dropout)

In [78]:
hidden_size = 500
dropout_rate = 0.25
weight_decay = 0.000001
learning_rate = 0.01

# Above inspired by last run best params, which were:
# {'lr': 0.0016359428701992723, 'weight_decay': 3.4162832861696797e-06, 'hidden_size': 493, 'dropout_rate': 0.4524812840729844}
model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()


# # Train the model with the best hyperparameters on the entire dataset
# for epoch in range(num_epochs):
#     ########################### Training #####################################
#     model.train(True)
#     train_loss = 0
#     count = 0
#     for batch, (X, y) in enumerate(train_loader):
#         X = X.float().to(device)
#         y = y.float().to(device)
#         prediction = model(X).to(device)
#         optimizer.zero_grad()
#         loss = loss_func(prediction, y.view(-1, 1).float().to(device))
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.float().item()
#         count += 1
#     scheduler.step()
#     print("Epoch", epoch+1, "Training loss:", train_loss / count)

# # Save the best model
# torch.save(model.state_dict(), "faceoffs_off_off_state_dict2.pt")
# # save the trained model
# torch.save(model, "faceoffs_off_off2.pt")

In [79]:
# Optuna Hyperparameter Tuning
import optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=trial_count)
print(study.best_params)

[I 2023-08-30 03:27:04,729] A new study created in memory with name: no-name-5d20ea02-d1ef-4a71-8234-71f3e799414d


learning rate: 6.809338136745198e-05
weight_decay: 1.7035177750901175e-05
hidden_size: 523
dropout_rate: 0.22678383656094042

EPOCH 1 of 50

Training loss: 0.005393321418927776
Validation loss: 0.0032931099024911723

EPOCH 2 of 50

Training loss: 0.005124916581230031
Validation loss: 0.003293322787309686

EPOCH 3 of 50

Training loss: 0.005166857037693262
Validation loss: 0.003307464144503077

EPOCH 4 of 50

Training loss: 0.0055441259820428156
Validation loss: 0.0033193297373751798

EPOCH 5 of 50

Training loss: 0.00522795661042134
Validation loss: 0.003344061862056454

EPOCH 6 of 50

Training loss: 0.005567361393736469
Validation loss: 0.0033532873882601657

EPOCH 7 of 50

Training loss: 0.005369110705537928
Validation loss: 0.003356541351725658

EPOCH 8 of 50

Training loss: 0.0052971851028915905
Validation loss: 0.00335526194733878

EPOCH 9 of 50

Training loss: 0.005238724820729759
Validation loss: 0.003342749628548821

EPOCH 10 of 50

Training loss: 0.0055675700099931825
Validati

[I 2023-08-30 03:30:57,631] Trial 0 finished with value: 0.003348852740600705 and parameters: {'lr': 6.809338136745198e-05, 'weight_decay': 1.7035177750901175e-05, 'hidden_size': 523, 'dropout_rate': 0.22678383656094042}. Best is trial 0 with value: 0.003348852740600705.


Validation loss: 0.003348852740600705
learning rate: 5.037295587384018e-05
weight_decay: 6.286699943283631e-06
hidden_size: 523
dropout_rate: 0.22708097259014226

EPOCH 1 of 50

Training loss: 0.0052463581992520224
Validation loss: 0.0033474373631179333

EPOCH 2 of 50

Training loss: 0.00541510350174374
Validation loss: 0.0033584447422375283

EPOCH 3 of 50

Training loss: 0.005220594401988719
Validation loss: 0.0033995783111701408

EPOCH 4 of 50

Training loss: 0.005152616054854459
Validation loss: 0.003449534997344017

EPOCH 5 of 50

Training loss: 0.00513926986604929
Validation loss: 0.003478188688556353

EPOCH 6 of 50

Training loss: 0.005177199504234725
Validation loss: 0.003495599919309219

EPOCH 7 of 50

Training loss: 0.005042081052023504
Validation loss: 0.003501219985385736

EPOCH 8 of 50

Training loss: 0.005585541483014822
Validation loss: 0.0034787846573938928

EPOCH 9 of 50

Training loss: 0.005247405523227321
Validation loss: 0.003491507920747002

EPOCH 10 of 50

Training

[I 2023-08-30 03:35:00,448] Trial 1 finished with value: 0.0034989824052900076 and parameters: {'lr': 5.037295587384018e-05, 'weight_decay': 6.286699943283631e-06, 'hidden_size': 523, 'dropout_rate': 0.22708097259014226}. Best is trial 0 with value: 0.003348852740600705.


...

Validation loss: 0.0033735747759540877
learning rate: 0.0001348845812356841
weight_decay: 4.1513019516844074e-05
hidden_size: 315
dropout_rate: 0.48675193334635625

EPOCH 1 of 50

Training loss: 0.005432782617087166
Validation loss: 0.003418881446123123

EPOCH 2 of 50

Training loss: 0.005673334209455384
Validation loss: 0.003504018997773528

EPOCH 3 of 50

Training loss: 0.005335816803077857
Validation loss: 0.003561601818849643

EPOCH 4 of 50

Training loss: 0.005544333304795954
Validation loss: 0.003580440611888965

EPOCH 5 of 50

Training loss: 0.005643016555243068
Validation loss: 0.0036290721812595925

EPOCH 6 of 50

Training loss: 0.005350824290265639
Validation loss: 0.0036069457419216633

EPOCH 7 of 50

Training loss: 0.005466588772833347
Validation loss: 0.0036176247522234917

EPOCH 8 of 50

Training loss: 0.00531235792570644
Validation loss: 0.0036692882422357798

EPOCH 9 of 50

Training loss: 0.005561589522080289
Validation loss: 0.003630140796303749

EPOCH 10 of 50

Traini

[I 2023-08-30 06:52:23,323] Trial 48 finished with value: 0.0035941048990935087 and parameters: {'lr': 0.0001348845812356841, 'weight_decay': 4.1513019516844074e-05, 'hidden_size': 315, 'dropout_rate': 0.48675193334635625}. Best is trial 16 with value: 0.0032828766076515117.


Validation loss: 0.0035941048990935087
learning rate: 1.958415748336568e-05
weight_decay: 0.0002497017971682417
hidden_size: 388
dropout_rate: 0.45871954394896586

EPOCH 1 of 50

Training loss: 0.005350134231977993
Validation loss: 0.0032970663936187825

EPOCH 2 of 50

Training loss: 0.005428922776546743
Validation loss: 0.003314737152929107

EPOCH 3 of 50

Training loss: 0.005383187879083885
Validation loss: 0.003333374004190167

EPOCH 4 of 50

Training loss: 0.00523840374727216
Validation loss: 0.0033469763584434986

EPOCH 5 of 50

Training loss: 0.005256647347576088
Validation loss: 0.0033555838745087385

EPOCH 6 of 50

Training loss: 0.005349032632592652
Validation loss: 0.0033658319928993783

EPOCH 7 of 50

Training loss: 0.00533243571408093
Validation loss: 0.003363782074302435

EPOCH 8 of 50

Training loss: 0.005259212080596222
Validation loss: 0.003356638985375563

EPOCH 9 of 50

Training loss: 0.005223029707041051
Validation loss: 0.003351263701915741

EPOCH 10 of 50

Training

[I 2023-08-30 06:56:26,085] Trial 49 finished with value: 0.003347292309626937 and parameters: {'lr': 1.958415748336568e-05, 'weight_decay': 0.0002497017971682417, 'hidden_size': 388, 'dropout_rate': 0.45871954394896586}. Best is trial 16 with value: 0.0032828766076515117.


Validation loss: 0.003347292309626937
{'lr': 0.00022213513827246681, 'weight_decay': 0.00011222525082498122, 'hidden_size': 305, 'dropout_rate': 0.4318767432591459}


In [80]:
best_params = study.best_trial.params
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

num_epochs = 50 # training for more epochs on best model (which will be saved)

train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

# Train the model with the best hyperparameters on the entire dataset
# for epoch in range(num_epochs):
#     ########################### Training #####################################
#     model.train(True)
#     train_loss = 0
#     count = 0
#     for batch, (X, y) in enumerate(train_loader):
#         X = X.float().to(device)
#         y = y.float().to(device)
#         prediction = model(X).to(device)
#         optimizer.zero_grad()
#         loss = loss_func(prediction, y.view(-1, 1).float().to(device))
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.float().item()
#         count += 1
#     print("Epoch", epoch+1, "Training loss:", train_loss / count)
#     scheduler.step()
#     model.eval()
#     val_loss = 0
#     count = 0
#     with torch.no_grad():
#         for (X, y) in valid_loader:
#             X = X.to(device)
#             y = y.to(device)
#             prediction = model(X).to(device)
#             loss = loss_func(prediction, y.view(-1, 1))
#             val_loss += loss.float().item()
#             count += val_batch_size
#         print("Validation loss:", val_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoff_mod_state_dict.pt")
# save the trained model
torch.save(model, "faceoff_mod.pt")

{'lr': 0.00022213513827246681, 'weight_decay': 0.00011222525082498122, 'hidden_size': 305, 'dropout_rate': 0.4318767432591459}

EPOCH 1 of 50

Training loss: 0.005767803561563293
Validation loss: 0.0033256261764715114

EPOCH 2 of 50

Training loss: 0.005680902447137568
Validation loss: 0.00336908937121431

EPOCH 3 of 50

Training loss: 0.005485808838986688
Validation loss: 0.003386912246545156

EPOCH 4 of 50

Training loss: 0.005497760579196943
Validation loss: 0.00340434528576831

EPOCH 5 of 50

Training loss: 0.005583293477280272
Validation loss: 0.0034163009840995073

EPOCH 6 of 50

Training loss: 0.005202007479965687
Validation loss: 0.0034341432619839907

EPOCH 7 of 50

Training loss: 0.005272911990889245
Validation loss: 0.003410453675314784

EPOCH 8 of 50

Training loss: 0.005360884178015921
Validation loss: 0.003407518845051527

EPOCH 9 of 50

Training loss: 0.005456977006461885
Validation loss: 0.0034053342727323375

EPOCH 10 of 50

Training loss: 0.005169595281283061
Validati

In [81]:
test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for batch, (X, y) in enumerate(test_loader):
        X = X.to(device)
        y = y.to(device)
        #print("" + str(X) + "" + str(y))
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print("Loss:", test_loss)
print(f"R-squared Coefficient: {r2:.4f}")

y_pred = np.array(y_pred)
y_true = np.array(y_true)

mean = data_unnorm['FA_zone_time'].mean()
sd = data_unnorm['FA_zone_time'].std()
print(mean)
print(sd)

# Calculate the unnormalized predictions and true labels
y_pred_unnorm = mean + sd * y_pred
y_true_unnorm = mean + sd * y_true

print("True labels (unnormalized):", y_true_unnorm)
print("Predictions (unnormalized):", y_pred_unnorm)
print("Loss:", test_loss)
print(f"R-squared Coefficient: {r2:.4f}")

data = {
    'player_id': player_ids_test,
    'opposing_player_id': opposing_player_ids_test,
    'season': seasons_test,
    'y_pred': y_pred,
    'y_pred_unnorm': y_pred_unnorm,
    'y_true': y_true,
    'y_true_unnorm': y_true_unnorm
}

# Create a DataFrame from the dictionary
summary_df = pd.DataFrame(data)

# Print the summary DataFrame
print(summary_df)
summary_df.to_csv("neural_net_faceoff_preds.csv")



Test Loss: 1.0074
True labels: [-0.30201727, 1.2703346, 0.58898216, -0.82613456, -0.45925248, -0.354429, 0.22210003, -0.40684074, -0.30201727, -0.82613456, 0.3269235, -0.09237035, -0.30201727, -0.56407595, -0.5116642, -0.45925248, -0.6164877, -0.14478208, -0.82613456, -0.56407595, -0.82613456, -0.7737228, -0.66889936, -0.5116642, -0.45925248, -0.5116642, -0.82613456, -0.82613456, -0.7213111, -0.56407595, 0.27451175, 0.22210003, 1.6372167, -0.5116642, -0.45925248, -0.19719382, 0.5365704, 3.2095687, -0.30201727, -0.40684074, 0.22210003, -0.354429, -0.40684074, 0.58898216, 4.0481563, -0.7213111, -0.56407595, -0.14478208, -0.45925248, -0.19719382, 0.1696883, -0.5116642, -0.56407595, -0.82613456, -0.40684074, -0.039958622, -0.24960554, -0.14478208, -0.24960554, -0.82613456, 0.11727657, 1.6372167, -0.66889936, -0.7213111, -0.66889936, -0.19719382, -0.45925248, 1.3751581, -0.039958622, -0.6164877, 2.2137458, -0.56407595, -0.09237035, 0.11727657, 0.37933522, -0.82613456, 0.27451175, -0.1971938

In [15]:
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

num_epochs = 50 # training for more epochs on best model (which will be saved)

train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)


{'lr': 0.05079528480724134, 'weight_decay': 0.0002664512798888168, 'hidden_size': 371, 'dropout_rate': 0.30519095473457875}

EPOCH 1 of 50

Training loss: 0.6805602010091146
Validation loss: 721.4608125

EPOCH 2 of 50

Training loss: 0.5989303792317708
Validation loss: 19.4476015625

EPOCH 3 of 50

Training loss: 0.4945974324544271
Validation loss: 1.871621826171875

EPOCH 4 of 50

Training loss: 0.3958716735839844
Validation loss: 1.329352294921875

EPOCH 5 of 50

Training loss: 0.44238685099283853
Validation loss: 0.656499755859375

EPOCH 6 of 50

Training loss: 0.4221099853515625
Validation loss: 0.40468789672851563

EPOCH 7 of 50

Training loss: 0.34276007588704427
Validation loss: 0.3281842041015625

EPOCH 8 of 50

Training loss: 0.41156744384765626
Validation loss: 0.3112650451660156

EPOCH 9 of 50

Training loss: 0.4296044413248698
Validation loss: 0.3059061279296875

EPOCH 10 of 50

Training loss: 0.39637403361002604
Validation loss: 0.30223544311523437

EPOCH 11 of 50

Trainin

0.30015359497070315

In [95]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_new = pd.read_csv("final_dataset_2022-2023_imputed.csv")

print(data_new.shape)

subset_data = data_new.iloc[:, include_col_indices].fillna(0)
df_subset = pd.DataFrame(subset_data)

player_ids_projections = data_new['event_player_1']
opposing_player_ids_projections = data_new['event_player_2']
seasons_projections = data_new['season']

data_new = df_subset.select_dtypes(include=['number'])

print(data_new.shape)
data_new_scaled = pd.DataFrame(scaler_fit.transform(data_new.values), columns = data_new.columns)

y = data_new['FA_zone_time']
x = data_new.loc[:, data_new.columns != 'FA_zone_time']
x = x.loc[:, x.columns != 'FA_zone_time']

projections_df = pd.concat([x, y], axis = 1)
projections_dataset = CustomDataset(projections_df)
projections_loader = DataLoader(projections_dataset, batch_size=test_batch_size, shuffle=False)

test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for batch, (X, y) in enumerate(projections_loader):
        X = X.to(device)
        y = y.to(device)
        #print("" + str(X) + "" + str(y))
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print("Loss:", test_loss)
print(f"R-squared Coefficient: {r2:.4f}")

y_pred = np.array(y_pred)
y_true = np.array(y_true)

mean = data_unnorm['FA_zone_time'].mean()
sd = data_unnorm['FA_zone_time'].std()
print(mean)
print(sd)

# Calculate the unnormalized predictions and true labels
y_pred_unnorm = mean + sd * y_pred
y_true_unnorm = mean + sd * y_true

print("True labels (unnormalized):", y_true_unnorm)
print("Predictions (unnormalized):", y_pred_unnorm)
print("Loss:", test_loss)
print(f"R-squared Coefficient: {r2:.4f}")

new_df = {
    'player_id': player_ids_projections,
    'opposing_player_id': opposing_player_ids_projections,
    'season': seasons_projections,
    'y_pred': y_pred,
    'y_pred_unnorm': y_pred_unnorm,
    'y_true': y_true,
    'y_true_unnorm': y_true_unnorm
}

# Create a DataFrame from the dictionary
projections_summary_df = pd.DataFrame(new_df)

# Print the summary DataFrame
print(projections_summary_df)
projections_summary_df.to_csv("neural_net_faceoff_projections.csv")

(833, 694)
(833, 310)
Test Loss: 497308586.6667
True labels: [40.0, 32.0, 7.0, 26.0, 9.0, 23.0, 14.0, 60.0, 5.0, 38.0, 30.0, 4.0, 28.0, 33.0, 4.0, 10.0, 28.0, 5.0, 20.0, 10.0, 16.0, 32.0, 7.0, 3.0, 9.0, 15.0, 8.0, 3.0, 10.0, 7.0, 19.0, 13.0, 10.0, 16.0, 7.0, 28.0, 5.0, 9.0, 3.0, 9.0, 5.0, 10.0, 4.0, 7.0, 3.0, 11.0, 13.0, 48.0, 22.0, 23.0, 13.0, 19.0, 8.0, 8.0, 19.0, 5.0, 12.0, 15.0, 65.0, 14.0, 18.0, 2.0, 44.0, 22.0, 3.0, 15.0, 9.0, 94.0, 8.0, 82.0, 11.0, 4.0, 30.0, 4.0, 9.0, 3.0, 19.0, 16.0, 7.0, 22.0, 53.0, 7.0, 5.0, 27.0, 11.0, 7.0, 18.0, 14.0, 26.0, 19.0, 12.0, 12.0, 22.0, 7.0, 15.0, 35.0, 12.0, 1.0, 3.0, 3.0, 13.0, 17.0, 18.0, 3.0, 7.0, 6.0, 5.0, 2.0, 10.0, 4.0, 8.0, 4.0, 14.0, 22.0, 29.0, 3.0, 5.0, 2.0, 7.0, 23.0, 12.0, 40.0, 9.0, 3.0, 30.0, 19.0, 19.0, 7.0, 19.0, 7.0, 67.0, 63.0, 6.0, 1.0, 9.0, 51.0, 22.0, 18.0, 22.0, 4.0, 12.0, 21.0, 7.0, 3.0, 12.0, 5.0, 18.0, 17.0, 12.0, 5.0, 4.0, 1.0, 4.0, 22.0, 5.0, 9.0, 16.0, 5.0, 61.0, 25.0, 14.0, 12.0, 15.0, 15.0, 22.0, 4.0, 1.0, 4.0, 40.